In [ ]:
%matplotlib inline

In [ ]:
import os
import glob
import json
import datetime
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
def objFromFile(filename):
    with open(filename, 'r') as fobj:
        json_str = fobj.readline()
        fobj.close()
    return json.loads(json_str)

In [ ]:
data_home = os.path.expanduser('~/Documents/Tesla')
folders = glob.glob('{}/2*'.format(data_home))
folders.sort()

t = []
d = []
for folder in folders:
    files = glob.glob('{}/*.json'.format(folder))
    files.sort()
    dd = []
    for file in files:
        #print('{}'.format(file))
        data = objFromFile(file)
        dd.append(data)
    d.append(dd)
    o = d[-1][-1]['charge_state']
    t.append(datetime.datetime.fromtimestamp(o['timestamp'] / 1000))
    print('{} -> {}% {} {}'.format(os.path.basename(file), o['battery_level'], o['charging_state'], t[-1].weekday()))

In [ ]:
# Find the previous Sunday
t0 = t[0]
oneday = datetime.timedelta(days=1)
while t0.weekday() != 6:
    t0 -= oneday
# display('{}'.format(t0.strftime('%Y/%m/%d %H:%M %A')))

# The first Sunday
t0 = datetime.datetime.strptime(t0.strftime('%Y-%m-%d'), '%Y-%m-%d')
display('{}'.format(t0.strftime('%Y/%m/%d %H:%M %A')))

# Group the weeks
tt = []
dd = []
k = 0
# while k < len(t):
for i in range(4):
    tw = []
    dw = []
    for i in range(7):
        if k < len(t) and t0 <= t[k] and t[k] < (t0 + oneday):
            tw.append(t[k])
            dw.append(d[k])
            k += 1
        else:
            tw.append(t0)
            dw.append(None)
        t0 += oneday
    tt.append(tw)
    dd.append(dw)
display(tt)

In [ ]:
import font

# bgColor = (1.0, 1.0, 1.0)
bgColor = (0.9, 0.9, 0.92)
o = 0.04

class WeeklyChart:
    def __init__(self, dpi=72, figsize=(900, 600), t=1.0):
        self._dpi = dpi
        self._figsize = figsize
        self.featureScale = t
        self.fontproperties = font.Properties(scale=t)

        # Use a separate set of context properties
        context_properties = {
            'font.family': 'serif',
            'font.serif': ['Arial', 'DejaVu Serif'],
            'axes.edgecolor': 'white',
            'axes.facecolor': 'black',
            'axes.labelcolor': 'white',
            'axes.linewidth': 1.0 * self.featureScale,
            'axes.titlepad': 8.0 * self.featureScale,
            'grid.color': 'white',
            'hatch.color': 'white',
            'text.color': 'white',
            'xtick.color': 'white',
            'xtick.direction': 'in',
            'xtick.major.pad': 9.0 * self.featureScale,
            'xtick.major.size': 4.0 * self.featureScale,
            'xtick.major.width': 1.0 * self.featureScale,
            'ytick.color': 'white',
            'ytick.direction': 'in',
            'ytick.major.pad': 7.5 * self.featureScale,
        }
        with matplotlib.pyplot.rc_context(context_properties):
            # Create a new figure
            w, h = self._figsize[0], self._figsize[1]    # Axis width & height in points
            figsize = np.array([w, h]) / self._dpi       # Figure size in inches
            self.fig = matplotlib.pyplot.figure(figsize=figsize, dpi=self._dpi, frameon=False)
            self.fig.patch.set_alpha(0.0)
            self.dat = matplotlib.pyplot.axes([0.0, 0.0, 1.0, 1.0], facecolor=None)
            self.dat.set_xlim((0, 7.0 + o))
            self.dat.set_ylim((0, 4.6 + o))
            self.dat.xaxis.set_visible(False)
            self.dat.yaxis.set_visible(False)
            self.dat.spines['top'].set_visible(False)
            self.dat.spines['right'].set_visible(False)
            self.dat.spines['bottom'].set_visible(False)
            self.dat.spines['left'].set_visible(False)
            self.dat.patch.set_visible(False)
            self.fig.canvas.draw()
        
        #ef = ImageFont.truetype('Symbola.ttf', size=36, encoding='unic')
        
        w = 1.0 - o
        h = 1.0 - o
        
        def compute_xr(x, w):
            return x + 0.96 * w
        
        y = 4.5
        label = self.dat.text(o, y, '{}'.format(tt[0][0].strftime('%B')),
                              fontproperties=matplotlib.font_manager.FontProperties(style='normal', size=28, weight='bold'),
                              horizontalalignment='left', verticalalignment='top')
        # Get the end of label
        box = label.get_window_extent().bounds
        self.dat.text(o + (box[0] + box[2]) / self._figsize[0] * (7.0 + o), y,
                      '{}'.format(tt[0][0].strftime('%Y')),
                      fontproperties=matplotlib.font_manager.FontProperties(style='normal', size=28, weight='normal'),
                      horizontalalignment='left', verticalalignment='top')

        self.dat.text(1.5, y, u'A\U0001F61EB',
                      fontproperties=matplotlib.font_manager.FontProperties(style='normal', size=28, weight='normal'),
                      horizontalalignment='left', verticalalignment='top')

        y -= 0.4
        for i in range(7):
            x = i + o
            xr = compute_xr(x, w)
            self.dat.text(xr, y, '{}'.format(tt[0][i].strftime('%a')),
                          fontproperties=matplotlib.font_manager.FontProperties(style='normal', size=16, weight='normal'),
                          horizontalalignment='right', verticalalignment='center')

        
        m = tt[0][0].month
        for j in range(len(tt)):
            for i in range(len(tt[j])):
                x = i + o
                y = 3.0 - j + o
                xc = x + 0.5 * w
                xr = compute_xr(x, w)
                
                # Outline of the frame
                rect = matplotlib.patches.Rectangle(xy=(x, y),
                                                    fill=False,
                                                    snap=True,
                                                    width=w,
                                                    height=h,
                                                    edgecolor='#999999',
                                                    linewidth=1.0,
                                                    alpha=1.0,
                                                    zorder=100)
                self.dat.add_patch(rect)

                # The day label.
                if m != tt[j][i].month:
                    m = tt[j][i].month
                    dayString = '{}'.format(tt[j][i].strftime('%b %-d'))
                else:
                    dayString = '{}'.format(tt[j][i].strftime('%-d'))
                self.dat.text(xr, y + 0.9, dayString,
                              fontproperties=matplotlib.font_manager.FontProperties(style='normal', size=14, weight='normal'),
                              horizontalalignment='right', verticalalignment='top')
                
                # Patch background
                rect = matplotlib.patches.Rectangle(xy=(x, y),
                                                    snap=True,
                                                    fill=True,
                                                    width=w,
                                                    height=h,
                                                    facecolor='#ffffff', zorder=1)
                self.dat.add_patch(rect)
                if dd[j][i] is None:
                    continue

                # Battery level
                color = '#88ff00'
                dayArray = dd[j][i]
                batteryLevel = dayArray[-1]['charge_state']['battery_level']
                if batteryLevel <= 60.0:
                    color = '#ffbb00'
                rect = matplotlib.patches.Rectangle(xy=(x, y),
                                                    fill=True,
                                                    snap=True,
                                                    width=w,
                                                    height=batteryLevel / 100 * h,
                                                    facecolor=color,
                                                    edgecolor=None,
                                                    alpha=0.7,
                                                    zorder=2)
                self.dat.add_patch(rect)
                self.dat.text(xc, y + 0.1, '{}'.format(tt[j][i].strftime('%I:%M %p')),
                             fontproperties=matplotlib.font_manager.FontProperties(style='normal', size=14, weight='normal'),
                             horizontalalignment='center', verticalalignment='bottom')
                self.dat.text(xc, y + 0.24, '({})'.format(len(dayArray)),
                             fontproperties=matplotlib.font_manager.FontProperties(style='normal', size=11, weight='normal'),
                             horizontalalignment='center', verticalalignment='bottom')
                self.dat.text(xc, y + 0.5, '{}%'.format(batteryLevel),
                             fontproperties=matplotlib.font_manager.FontProperties(style='normal', size=24, weight='bold'),
                             color='#000000',
                             horizontalalignment='center', verticalalignment='center')


In [ ]:
view = WeeklyChart()

In [ ]:
fig = plt.figure(figsize=(12, 3), dpi=96)
plt.subplots_adjust(bottom=0.2)
plt.xticks(rotation=35)
ax = plt.gca()
xfmt = matplotlib.dates.DateFormatter('%Y-%m-%d')
ax.xaxis.set_major_formatter(xfmt)
# plt.plot(t, b, '.-')
plt.stem(t, b)
plt.grid()

In [ ]:
t = []
b = []
for file in files:
    data = objFromFile(file)
    o = data['drive_state']
    
    o = data['charge_state']
    t.append(datetime.datetime.fromtimestamp(o['timestamp'] / 1000))
    b.append(o['battery_level'])
    print('{} -> {}% {}'.format(os.path.basename(file), o['battery_level'], o['charging_state']))

In [ ]:
fig = plt.figure(figsize=(12, 3), dpi=96)
plt.subplots_adjust(bottom=0.2)
plt.xticks(rotation=25)
ax = plt.gca()
xfmt = matplotlib.dates.DateFormatter('%Y-%m-%d %H:%M:%S')
ax.xaxis.set_major_formatter(xfmt)
# plt.plot(t, b, '.-')
plt.stem(t, b)

In [ ]:
for file in files[-5:]:
    data = objFromFile(file)
    display(file)
    display(data['drive_state'])